In [1]:
import pandas as pd
from io import StringIO
import os

In [2]:
# df = pd.read_pickle("./aggregate_data.pkl")

In [3]:
def process_lbl(line_list, file): 
    name_dict = {}
    data_dict = {}
    name =''
    i = 0
    CSV = ["Filename,Application,Config,Is_Match,Testcase,Edges,States,Component,test,subtest,metric,metric_type"]
    while (i < len(line_list)):
        if i+1 < len(line_list) and line_list[i+1] == '---------,---------,---------,---------\n':
            firstTwoCols, edges, states = line_list[i].strip().rsplit(",", 2)
            name, regex = firstTwoCols.split(",", 1)
            i = i + 1
        elif line_list[i] == '---------,---------,---------,---------\n':
            i = i + 1
        elif line_list[i] != 'Component,test,subtest,metric,metric_type\n':
            CSV.append("{},{},{},{},{}".format(file,name,edges,states,line_list[i]))
            i = i + 1
        else:
            i = i + 1
    return pd.read_csv(StringIO("\n".join(CSV)))

In [4]:
# Filename -> "test, configuration, is_match?"
filemap = {
    "email_dkim": ["Email", "Reef", True],
    "email_dkim_nwr": ["Email", "DFA+recursion", True],
    "email_dkim_hybrid": ["Email", "SAFA+lookup", True],
    "email_dkim_naive": ["Email", "DFA", True],
    "pihole": ["oDoH", "reef", True],
    "pihole_hybrid": ["oDoH", "SAFA+lookup", True],
    "pihole_nwr": ["oDoH", "DFA+recursion", True],
    # "pihole_naive": ["oDoH", "DFA", True),
    "good_pass": ["password", "Reef", True],
    "bad_pass": ["password", "Reef", False],
    "good_pass_hybrid": ["password", "SAFA+lookup", True],
    "bad_pass_hybrid": ["password", "SAFA+lookup", False],
    "brca2_var1_match_proj": ["DNA", "Reef", True],
    "brca2_primary_nonmatch_proj": ["DNA", "Reef", False]
}

In [5]:
def parse_file(fn):
    if fn in filemap:
        with open(fn, 'r') as f:
            prefix = ','.join([fn] + [str(item) for item in filemap.get(fn)])
            return process_lbl(f.readlines(), prefix)
    else:
        pd.DataFrame()

In [6]:
df = pd.DataFrame()
for fn in os.listdir():
    df = pd.concat([df, parse_file(fn)], ignore_index=True)
        

In [87]:
parse_file("good_pass_hybrid")

,Filename,Application,Config,Is_Match,Testcase,Edges,States,Component,test,subtest,metric,metric_type
0,good_pass_hybrid,password,SAFA+lookup,True,B6u$r@s#R5_12,1188,21,prover,prove,prove_3,547305,μs
1,good_pass_hybrid,password,SAFA+lookup,True,B6u$r@s#R5_12,1188,21,prover,Prover,Full,6002712,μs
2,good_pass_hybrid,password,SAFA+lookup,True,B6u$r@s#R5_12,1188,21,solver,Witness Generation,step_0,25597,μs
3,good_pass_hybrid,password,SAFA+lookup,True,B6u$r@s#R5_12,1188,21,compiler,R1CS,Proof Setup,8077595,μs
4,good_pass_hybrid,password,SAFA+lookup,True,B6u$r@s#R5_12,1188,21,solver,Witness Generation,step_3,32055,μs
...,...,...,...,...,...,...,...,...,...,...,...,...
253,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,verifier,Verification,Final Checks Consistency Verification,126352,μs
254,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,solver,Witness Generation,step_0,13384,μs
255,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,prover,add test,Primary Circuit,19429,constraints
256,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,prover,add test,Secondary Circuit,10347,constraints


In [7]:
df

,Filename,Application,Config,Is_Match,Testcase,Edges,States,Component,test,subtest,metric,metric_type
0,brca2_primary_nonmatch_proj,DNA,Reef,False,./tests/do_32325508,4857,976,compiler,Compiler,Full,70055313,μs
1,brca2_primary_nonmatch_proj,DNA,Reef,False,./tests/do_32325508,4857,976,verifier,Verification,Nova Verification,2182705,μs
2,brca2_primary_nonmatch_proj,DNA,Reef,False,./tests/do_32325508,4857,976,compiler,R1CS,R1CS precomputations,2889874,μs
3,brca2_primary_nonmatch_proj,DNA,Reef,False,./tests/do_32325508,4857,976,compiler,SAFA,SAFA,13420651,μs
4,brca2_primary_nonmatch_proj,DNA,Reef,False,./tests/do_32325508,4857,976,compiler,R1CS,To Circuit,21683,μs
...,...,...,...,...,...,...,...,...,...,...,...,...
9305,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,verifier,Verification,Final Checks Consistency Verification,126352,μs
9306,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,solver,Witness Generation,step_0,13384,μs
9307,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,prover,add test,Primary Circuit,19429,constraints
9308,good_pass_hybrid,password,SAFA+lookup,True,fpVz4W#$VB_12,1188,21,prover,add test,Secondary Circuit,10347,constraints


In [9]:
df["test"].unique()

array(['Compiler', 'Verification', 'R1CS', 'SAFA', 'Proof', 'Prover',
       'prove', 'Solve', 'Witness Generation', 'add test', 'Proof Size',
       'DFA', 'DFA Solving', 'Witness', 'Prove', 'Verifier'], dtype=object)